In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import seaborn as sns
from pandas import ExcelWriter
from pandas import ExcelFile
import sample_utils

%matplotlib inline

In [14]:
file_path = '2020_LER_20201102_testset_V04.xlsx'

real_seed_list = [990, 8277, 8904, 1199, 4128, 2922, 706, 8754, 2212, 7733]
gen_seed_list = [9226,  807,  524, 2646, 1638, 3741,  328, 7822, 9461, 8437]

true_type = 'b'

# OPTIMAL

In [11]:
op_EMD_total = []

for i in range(10):
    
    file_path = '2020_LER_20201102_testset_V04.xlsx'

    real_seed = '_seed_{}'.format(real_seed_list[i])

    # data_A

    data = np.load('../data_handler/'+file_path+'_data_a'+real_seed+'.npy', allow_pickle=True)
    num_in_cycle_half = [125]*10
    num_of_cycle = 10

    A_X_all, A_Y_all, A_X_per_cycle, A_Y_per_cycle = np.array(data[0]), np.array(data[1]), np.array(data[2]), np.array(data[3])

    print(A_X_all.shape, A_Y_all.shape, A_X_per_cycle.shape, A_Y_per_cycle.shape)
    print()
    A_test_real = A_Y_all

    # for pair plot indexing
    A_test_X_cycle = A_X_per_cycle

    A_y_mean = np.mean(A_Y_all, axis=0, dtype=np.float32)
    A_y_std = np.std(A_Y_all, axis=0, dtype=np.float32)

    print("!! check !! test real shape", A_test_real.shape)
    print("!! check !! X value ( especially its precision )")
    print()

    # data_B

    data = np.load('../data_handler/'+file_path+'_data_b'+real_seed+'.npy', allow_pickle=True)

    B_X_all, B_Y_all, B_X_per_cycle, B_Y_per_cycle = np.array(data[0]), np.array(data[1]), np.array(data[2]), np.array(data[3])
    print(B_X_all.shape, B_Y_all.shape, B_X_per_cycle.shape, B_Y_per_cycle.shape)
    print()
    B_test_real = B_Y_all

    # for pair plot indexing
    B_test_X_cycle = B_X_per_cycle

    B_y_mean = np.mean(B_Y_all, axis=0, dtype=np.float32)
    B_y_std = np.std(B_Y_all, axis=0, dtype=np.float32)

    print("!! check !! test real shape", B_test_real.shape)
    print("!! check !! X value ( especially its precision )")

    ### min, max
    
    min1_per_out_v = A_test_real.min(axis=0)
    max1_per_out_v = A_test_real.max(axis=0)
    print("minimum y value for each varaible:", min1_per_out_v)
    print("maximum y value for each varaible:", max1_per_out_v)

    min2_per_out_v = B_test_real.min(axis=0)
    max2_per_out_v = B_test_real.max(axis=0)
    print("minimum y value for each varaible:", min2_per_out_v)
    print("maximum y value for each varaible:", max2_per_out_v)

    min_per_out_v = np.minimum(min1_per_out_v, min2_per_out_v)
    max_per_out_v = np.maximum(max1_per_out_v, max2_per_out_v)
    
    # EMD  

    EMD_1d_score_list, EMD_2d_score_list = sample_utils.EMD_all_pair_each_X_integral(generated_samples = A_test_real, real_samples = B_test_real, num_coordinate=10, num_of_cycle=num_of_cycle, num_in_gen_list=num_in_cycle_half, num_in_real_list = num_in_cycle_half, min_list = min_per_out_v, max_list = max_per_out_v) 

    print(EMD_1d_score_list.shape)
    print(EMD_2d_score_list.shape)

    EMD_score_cat = np.hstack((EMD_1d_score_list, EMD_2d_score_list))

    EMD_score = np.mean(EMD_score_cat)
    EMD_score_list_integral = np.mean(EMD_score_cat,axis=1)
    EMD_std = np.sqrt(np.mean((EMD_score - EMD_score_list_integral)**2))

#     for i in range(len(EMD_score_list_integral)):
#         print("EMD_score for each X factor {} \t score {:5f} \t dev {:5f}".format(i, EMD_score_list_integral[i], (EMD_score_list_integral[i]-EMD_score)/EMD_std))
    # A: Ioff, B: IDSat, C: IDlin, D: VTsat, E:VTlin, F:SS, G: classs

    EMD_score_list = np.array(EMD_score_list_integral)
    print("EMD score: ", EMD_score)
    print("EMD argsort: ", EMD_score_list.argsort())
    
    op_EMD_total.append(EMD_score_list)


(1250, 5) (1250, 6) (10, 5) (10, 6)

!! check !! test real shape (1250, 6)
!! check !! X value ( especially its precision )

(1250, 5) (1250, 6) (10, 5) (10, 6)

!! check !! test real shape (1250, 6)
!! check !! X value ( especially its precision )
minimum y value for each varaible: [5.2280e-13 3.1240e-05 1.2690e-05 3.1900e-01 3.3200e-01 6.0799e+01]
maximum y value for each varaible: [8.3370e-12 4.5520e-05 1.8750e-05 3.7600e-01 3.8100e-01 6.3893e+01]
minimum y value for each varaible: [3.0220e-13 2.6090e-05 1.1500e-05 3.1800e-01 3.3000e-01 6.0765e+01]
maximum y value for each varaible: [8.2640e-12 4.5900e-05 1.8520e-05 3.9400e-01 3.9700e-01 6.3635e+01]
(10, 6)
(10, 15)
EMD score:  0.14933017158792133
EMD argsort:  [9 8 6 2 1 7 3 5 0 4]
(1250, 5) (1250, 6) (10, 5) (10, 6)

!! check !! test real shape (1250, 6)
!! check !! X value ( especially its precision )

(1250, 5) (1250, 6) (10, 5) (10, 6)

!! check !! test real shape (1250, 6)
!! check !! X value ( especially its precision )
minim

In [12]:
# EMD mean, std

op_EMD_total_np = np.array(op_EMD_total)

print(op_EMD_total_np)

print(np.mean(op_EMD_total_np, axis=0))
print(np.std(op_EMD_total_np, axis=0))

[[0.22996005 0.12680881 0.12317108 0.1986176  0.26572101 0.2040112
  0.08956111 0.15846997 0.05805591 0.03892498]
 [0.09084723 0.12180568 0.13226578 0.25023782 0.17982312 0.31898773
  0.04258873 0.12459171 0.07838362 0.02185755]
 [0.1098181  0.12992462 0.10827315 0.29113902 0.11824719 0.16340381
  0.02501009 0.13223614 0.04972934 0.10186569]
 [0.31220551 0.19240443 0.10622243 0.16044441 0.27151049 0.11137719
  0.03216477 0.15291012 0.08550628 0.03987737]
 [0.12112328 0.57779732 0.10740572 0.14016646 0.13426047 0.10747856
  0.02454601 0.20206262 0.06924995 0.034829  ]
 [0.16063376 0.25575477 0.07624958 0.11780218 0.44292962 0.19723453
  0.02120506 0.16652992 0.09544289 0.04948098]
 [0.2966943  0.16623857 0.29214104 0.2641028  0.19674689 0.14074897
  0.04493128 0.33430148 0.10228492 0.03164621]
 [0.23056313 0.10440979 0.11030654 0.11959696 0.19920361 0.12829443
  0.04584086 0.24769485 0.08840359 0.05027599]
 [0.18743101 0.14738432 0.07773589 0.18175312 0.24358638 0.27688269
  0.04769837 

# GAN

In [15]:
EMD_total = []

for i in range(10):
    
    real_seed = '_seed_{}'.format(real_seed_list[i])
    gen_seed = gen_seed_list[i]
    
    #### real data ####
    
    data = np.load('../data_handler/'+file_path+'_data_'+true_type+real_seed+'.npy', allow_pickle=True)
    num_in_cycle_half = [125]*10
    num_of_cycle = 10

    X_all, Y_all, X_per_cycle, Y_per_cycle = np.array(data[0]), np.array(data[1]), np.array(data[2]), np.array(data[3])

    print(X_all.shape, Y_all.shape, X_per_cycle.shape, Y_per_cycle.shape)
    print()
    test_real = Y_all

    # for pair plot indexing
    test_X_cycle = X_per_cycle

    y_mean = np.mean(Y_all, axis=0, dtype=np.float32)
    y_std = np.std(Y_all, axis=0, dtype=np.float32)

    print("!! check !! test real shape", test_real.shape)
    print("!! check !! X value ( especially its precision )")
    print()
    
    #### gen data ####
    
    result = 'test_specificnaive_date_210130_data_2020_LER_20201008_V008.xlsx_model_mlp_gan1_seed_{}_lr_5e-05_0.0001_0.0005_hidden_dim_100_50_batch_size_32_noise_d_100_sample_num_125_tr_num_in_cycle_50_pdrop_0.9.npy'.format(gen_seed)
    test_gen = np.load(result, allow_pickle=True)

    num_of_cycle = 10
    test_gen_sample_num = int(test_gen.shape[0]/num_of_cycle)

    print("test data sample num", test_gen_sample_num)
    print("test data shape", test_gen.shape)

    min1_per_out_v = test_real.min(axis=0)
    max1_per_out_v = test_real.max(axis=0)
    print("minimum y value for each varaible:", min1_per_out_v)
    print("maximum y value for each varaible:", max1_per_out_v)

    min2_per_out_v = test_gen.min(axis=0)
    max2_per_out_v = test_gen.max(axis=0)
    print("minimum y value for each varaible:", min2_per_out_v)
    print("maximum y value for each varaible:", max2_per_out_v)

    min_per_out_v = np.minimum(min1_per_out_v, min2_per_out_v)
    max_per_out_v = np.maximum(max1_per_out_v, max2_per_out_v)
    
    ### EMD ###
    
    EMD_1d_score_list, EMD_2d_score_list = sample_utils.EMD_all_pair_each_X_integral(generated_samples = test_gen, real_samples = test_real, num_coordinate=10, num_of_cycle=num_of_cycle, num_in_gen_list=[test_gen_sample_num]*num_of_cycle, num_in_real_list = num_in_cycle_half, min_list = min_per_out_v, max_list = max_per_out_v) 

    print(EMD_1d_score_list.shape)
    print(EMD_2d_score_list.shape)

    EMD_score_cat = np.hstack((EMD_1d_score_list, EMD_2d_score_list))

    EMD_score = np.mean(EMD_score_cat)
    EMD_score_list_integral = np.mean(EMD_score_cat,axis=1)
    EMD_std = np.sqrt(np.mean((EMD_score - EMD_score_list_integral)**2))

    for i in range(len(EMD_score_list_integral)):
        print("EMD_score for each X factor {} \t score {:5f} \t dev {:5f}".format(i, EMD_score_list_integral[i], (EMD_score_list_integral[i]-EMD_score)/EMD_std))
    # A: Ioff, B: IDSat, C: IDlin, D: VTsat, E:VTlin, F:SS, G: classs

    EMD_score_list = np.array(EMD_score_list_integral)
    print("EMD score: ", EMD_score)


    print("EMD argsort: ", EMD_score_list.argsort())
    
    EMD_total.append(EMD_score_list)
    print(EMD_total)

(1250, 5) (1250, 6) (10, 5) (10, 6)

!! check !! test real shape (1250, 6)
!! check !! X value ( especially its precision )

test data sample num 125
test data shape (1250, 6)
minimum y value for each varaible: [3.0220e-13 2.6090e-05 1.1500e-05 3.1800e-01 3.3000e-01 6.0765e+01]
maximum y value for each varaible: [8.2640e-12 4.5900e-05 1.8520e-05 3.9400e-01 3.9700e-01 6.3635e+01]
minimum y value for each varaible: [3.22169752e-13 2.97672097e-05 1.22695943e-05 3.19973155e-01
 3.32721367e-01 6.06838808e+01]
maximum y value for each varaible: [6.14563458e-12 4.49361305e-05 1.85497985e-05 3.76531002e-01
 3.82135944e-01 6.33229194e+01]
(10, 6)
(10, 15)
EMD_score for each X factor 0 	 score 0.182965 	 dev 0.222566
EMD_score for each X factor 1 	 score 0.269513 	 dev 1.404093
EMD_score for each X factor 2 	 score 0.145330 	 dev -0.291220
EMD_score for each X factor 3 	 score 0.276124 	 dev 1.494335
EMD_score for each X factor 4 	 score 0.254099 	 dev 1.193663
EMD_score for each X factor 5 	 sc

(10, 6)
(10, 15)
EMD_score for each X factor 0 	 score 0.276497 	 dev 0.457040
EMD_score for each X factor 1 	 score 0.531464 	 dev 2.427289
EMD_score for each X factor 2 	 score 0.221832 	 dev 0.034618
EMD_score for each X factor 3 	 score 0.207843 	 dev -0.073486
EMD_score for each X factor 4 	 score 0.322750 	 dev 0.814460
EMD_score for each X factor 5 	 score 0.175167 	 dev -0.325991
EMD_score for each X factor 6 	 score 0.087361 	 dev -1.004503
EMD_score for each X factor 7 	 score 0.162129 	 dev -0.426735
EMD_score for each X factor 8 	 score 0.126191 	 dev -0.704446
EMD_score for each X factor 9 	 score 0.062289 	 dev -1.198248
EMD score:  0.21735243906400034
EMD argsort:  [9 6 8 7 5 3 2 0 4 1]
[array([0.18296529, 0.26951328, 0.14532994, 0.2761236 , 0.25409906,
       0.15582706, 0.07145991, 0.13596899, 0.07968531, 0.09564859]), array([0.26751413, 0.16175351, 0.21197237, 0.23295041, 0.13654333,
       0.37070449, 0.10375005, 0.18784966, 0.12523248, 0.07597514]), array([0.3005946

(10, 6)
(10, 15)
EMD_score for each X factor 0 	 score 0.227651 	 dev -0.072908
EMD_score for each X factor 1 	 score 0.453759 	 dev 1.656997
EMD_score for each X factor 2 	 score 0.231593 	 dev -0.042742
EMD_score for each X factor 3 	 score 0.322984 	 dev 0.656465
EMD_score for each X factor 4 	 score 0.264943 	 dev 0.212405
EMD_score for each X factor 5 	 score 0.440952 	 dev 1.559014
EMD_score for each X factor 6 	 score 0.065670 	 dev -1.312184
EMD_score for each X factor 7 	 score 0.174978 	 dev -0.475889
EMD_score for each X factor 8 	 score 0.105416 	 dev -1.008099
EMD_score for each X factor 9 	 score 0.083854 	 dev -1.173061
EMD score:  0.2371799637891118
EMD argsort:  [6 9 8 7 0 2 4 3 5 1]
[array([0.18296529, 0.26951328, 0.14532994, 0.2761236 , 0.25409906,
       0.15582706, 0.07145991, 0.13596899, 0.07968531, 0.09564859]), array([0.26751413, 0.16175351, 0.21197237, 0.23295041, 0.13654333,
       0.37070449, 0.10375005, 0.18784966, 0.12523248, 0.07597514]), array([0.30059462

In [16]:
# EMD mean, std

EMD_total_np = np.array(EMD_total)

print(np.mean(EMD_total_np, axis=0))
print(np.std(EMD_total_np, axis=0))

[0.25891237 0.27224507 0.2083739  0.27938277 0.26947012 0.24967277
 0.07824968 0.2037121  0.13235742 0.09366655]
[0.03819877 0.12648315 0.04105239 0.0533906  0.06689044 0.0874736
 0.01441383 0.05771908 0.04028882 0.03193369]


# Linear Gaussian